<h1 style="Comic Sans MS; text-align: center; color: #FFC300">Regularized Linear Regression: US demographic and health data</h1>
<h3 id="step1" style="font-family: Comic Sans MS; color: #68FF33">1. Problem statement and data collection</h3>
<p style="color: pink">Sociodemographic and health resource data have been collected by county in the United States and we want to find out if there is any relationship between health resources and sociodemographic data. To do this, we have to:
</p>
<ul style="color: pink">
    <li>set a target variable (health-related) to conduct the analysis. In this case we select: <strong><em>Heart disease_number</em></strong></li>
    <li>perform EDA so we keep the variables that are strictly necessary and eliminate those that are not relevant or do not provide information.</li>
    <li>start solving the problem by implementing a linear regression model and analyze the results. Then, using the same data and default attributes, build a Lasso model and compare the results with the baseline linear regression.</li>
    <li>analyze how R-squared evolves when the hyperparameter of the Lasso model changes (for example we can start testing from a value of 0.0 and work our way up to a value of 20). Draw these values in a line diagram.</li>
    <li>last, after training the Lasso model, if the results are not satisfactory, optimize it using one of the techniques seen in the module.</li>
</ul>
<div class="alert alert-block alert-warning">
<b>IMPORTANT:</b> In developing this project, I adhered to and replicated all the steps outlined in the provided solution. However, throughout the process, I meticulously compared and contrasted each element, supplementing the solution with my own insights and commentary to demonstrate a comprehensive understanding of its nuances. Also I have performed <b>Ridge</b> linear regression in model optimization.
</div>

In [1]:
import pandas as pd

total_data = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv')
total_data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


<h3 id="step2" style="font-family: Comic Sans MS; color: #68FF33">2. Exploration and data cleaning</h3>
<p style="color: pink">We will conduct a concise Exploratory Data Analysis (EDA), focusing on essential aspects without delving into intricate details.</p>

In [3]:
total_data.shape

(3140, 108)

In [5]:
total_data = total_data.drop_duplicates().reset_index(drop=True)
# by specifying drop=True, the method discards the old index entirely and replaces it with the default integer index
total_data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


<h3 id="step3" style="font-family: Comic Sans MS; color: #68FF33">3. Feature engineering</h3>
<strong><span style="color: red">Normalization</span></strong>
 <p style="color: pink">Feature scaling is a crucial step in data preprocessing for many Machine Learning algorithms. It is a technique that changes the range of data values so that they can be compared to each other. Scaling usually involves normalization, which is the process of changing the values so that they have a mean of 0 and a standard deviation of 1.</p>

In [8]:
from sklearn.preprocessing import StandardScaler

data_types = total_data.dtypes
# data_types is a series with the column names as the index and the data types as the values
numeric_columns = [column for column in list(data_types[data_types != 'object'].index) if column != 'Heart disease_number']
# list comprehension to create a list of the column names that are not of type 'object' (so only numeric data) and are not the target variable 'Heart disease_number'
scaler = StandardScaler() # create a StandardScaler object
norm_features = scaler.fit_transform(total_data[numeric_columns]) # fit the scaler to the data and transform the data

total_data_scal = pd.DataFrame(norm_features, index = total_data.index, columns=numeric_columns) # create a dataframe from the scaled data
total_data_scal['Heart disease_number'] = total_data['Heart disease_number'] # add the target variable to the dataframe
total_data_scal.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,Heart disease_number
0,-1.940874,-0.145679,-0.142421,0.158006,-0.135556,0.573496,-0.153144,0.027610,-0.139384,0.588469,...,-0.063696,-0.071720,-0.089834,-0.129902,-0.609615,-0.582796,-0.669652,-0.147523,-1.082865,3345
1,-1.940742,0.341296,0.287476,-0.242861,0.320383,-0.193107,0.183774,-0.469965,0.230620,-0.110300,...,-0.394103,-0.414900,-0.337677,0.376251,-0.433549,-0.393279,-0.343373,0.389791,-0.420704,13414
2,-1.940610,-0.237785,-0.239429,-0.419441,-0.246181,-0.439718,-0.225971,0.272104,-0.218759,0.656538,...,2.432709,2.483064,2.317776,-0.183415,1.855312,1.880929,1.777443,-0.204321,0.903618,2159
3,-1.940478,-0.245223,-0.246032,-0.426966,-0.254791,-0.609076,-0.230792,0.396168,-0.220555,1.264959,...,0.376846,0.423984,0.299632,-0.229096,-0.257483,-0.203761,-0.180233,-0.242100,-1.745026,1533
4,-1.940346,-0.138966,-0.135053,0.186249,-0.137140,0.216679,-0.155888,-0.200808,-0.143570,0.088582,...,0.156575,0.195197,0.158008,-0.111247,-0.081417,-0.014244,-0.017093,-0.124105,-1.745026,4101


<strong><span style="color: red">Feature selection</span></strong>

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

X = total_data_scal.drop('Heart disease_number', axis=1) # create the feature matrix X by dropping the target variable. Can also be written as total_data_scal.drop(columns='Heart disease_number')
y = total_data_scal['Heart disease_number'] # create the target variable y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # split the data into training and testing sets
train_indices = list(X_train.index) # get the indices of the training set
test_indices = list(X_test.index) # get the indices of the testing set

k = int(len(X_train.columns) * 0.3) # select the top 30% of features
selection_model = SelectKBest(score_func=f_regression, k=k) # create a SelectKBest object 
selection_model.fit(X_train, y_train) # fit the model to the training data
ix = selection_model.get_support() # get the indices of the selected features

X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns=X_train.columns.values[ix]) # create a dataframe from the selected features
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns=X_test.columns.values[ix]) # create a dataframe from the selected features
X_train_sel.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Family Medicine/General Practice Primary Care (2019),Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,COPD_number,diabetes_number,CKD_number
0,-0.232556,-0.227731,-0.234284,-0.232951,-0.226353,-0.231316,-0.229599,-0.233425,-0.234680,-0.234420,...,-0.212643,-0.208590,-0.231195,-0.229737,-0.233171,-0.234370,-0.232975,-0.223516,-0.218609,-0.219329
1,-0.158676,-0.178665,-0.180166,-0.188266,-0.175070,-0.161168,-0.134688,-0.105618,-0.119270,-0.091822,...,-0.116680,-0.110850,-0.150293,-0.098866,-0.152859,-0.142645,-0.155304,-0.110080,-0.131449,-0.130962
2,-0.199114,-0.211128,-0.195138,-0.166782,-0.195036,-0.194045,-0.199725,-0.219256,-0.222207,-0.205154,...,-0.192263,-0.217668,-0.197005,-0.216056,-0.195125,-0.193205,-0.201976,-0.193106,-0.189197,-0.206391
3,-0.036595,-0.037734,-0.017077,-0.057986,-0.052252,-0.033158,-0.020228,-0.032603,-0.023876,-0.046224,...,0.062458,-0.107888,-0.036940,-0.030034,-0.039882,-0.003321,0.006163,-0.007077,-0.047515,-0.045054
4,0.090839,0.094680,0.101662,0.056721,0.042392,0.068095,0.101699,0.144664,0.140685,0.166099,...,0.274818,0.194913,0.097767,0.161314,0.088485,0.165555,0.182740,0.265603,0.123040,0.132454


In [10]:
X_test_sel.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Family Medicine/General Practice Primary Care (2019),Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,COPD_number,diabetes_number,CKD_number
0,-0.285286,-0.285362,-0.294836,-0.269566,-0.258568,-0.268541,-0.289649,-0.312989,-0.316763,-0.286734,...,-0.303292,-0.285225,-0.284324,-0.308211,-0.283698,-0.302439,-0.302292,-0.324038,-0.276290,-0.281172
1,0.496553,0.433072,0.392170,0.544659,0.453677,0.391480,0.499744,0.668639,0.716353,0.476084,...,0.853184,0.424904,0.477184,0.620724,0.517408,0.527360,0.516364,0.443806,0.418504,0.454092
2,-0.260191,-0.255123,-0.265837,-0.246628,-0.234723,-0.240703,-0.264552,-0.289867,-0.289846,-0.290962,...,-0.277451,-0.261868,-0.257294,-0.287868,-0.259943,-0.249299,-0.259877,-0.225107,-0.225970,-0.242229
3,0.039389,0.058341,0.059701,-0.018647,0.003236,0.030594,0.074401,0.091003,0.060721,0.005012,...,0.197267,0.130719,0.036299,0.055281,0.031494,0.035274,0.026108,0.136643,0.003409,0.022352
4,0.364272,0.281232,0.323623,0.525353,0.295990,0.288317,0.298029,0.461297,0.497760,0.351393,...,0.659217,0.305024,0.336581,0.423969,0.390596,0.271127,0.273318,0.329669,0.256620,0.334804


In [11]:
X_train_sel['Heart disease_number'] = list(y_train) # add the target variable to the dataframe
X_test_sel['Heart disease_number'] = list(y_test) # add the target variable to the dataframe
X_train_sel.to_csv('C:/Users/Jorge Payà/Desktop/4Geeks/DSML Bootcamp/Projects/Regularized-Linear-model/data/processed/clean_train.csv', index=False) # save the training data to a csv file
X_test_sel.to_csv('C:/Users/Jorge Payà/Desktop/4Geeks/DSML Bootcamp/Projects/Regularized-Linear-model/data/processed/clean_test.csv', index=False) # save the testing data to a csv file

<ul style="color: pink">
    <li>We add a new column named 'Heart disease_number' to the <b><em>X_train_sel</em></b> and <b><em>X_test_sel</em></b> which are taken from <b><em>y_train</em></b> and <b><em>y_test</em></b> respectively due to those contain the target values.</li>
    <li>The <b><em>list()</em></b> function is used to convert <b><em>y_train</em></b> and <b><em>y_test</em></b> into a list. This is necessary because when adding a new column to a DataFrame, the values need to be in a list-like structure (a list, a Series, an array, etc.).</li>
</ul>

<h3 id="step5" style="font-family: Comic Sans MS; color: #68FF33">5. Linear Regression: Model training and optimization</h3>
<strong><span style="color: red">Reading the processed dataset and plotting features</span></strong>

In [12]:
total_data = pd.concat([X_train_sel, X_test_sel]) # concatenate the training and testing data
total_data.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,COPD_number,diabetes_number,CKD_number,Heart disease_number
0,-0.232556,-0.227731,-0.234284,-0.232951,-0.226353,-0.231316,-0.229599,-0.233425,-0.234680,-0.234420,...,-0.208590,-0.231195,-0.229737,-0.233171,-0.234370,-0.232975,-0.223516,-0.218609,-0.219329,2072
1,-0.158676,-0.178665,-0.180166,-0.188266,-0.175070,-0.161168,-0.134688,-0.105618,-0.119270,-0.091822,...,-0.110850,-0.150293,-0.098866,-0.152859,-0.142645,-0.155304,-0.110080,-0.131449,-0.130962,3796
2,-0.199114,-0.211128,-0.195138,-0.166782,-0.195036,-0.194045,-0.199725,-0.219256,-0.222207,-0.205154,...,-0.217668,-0.197005,-0.216056,-0.195125,-0.193205,-0.201976,-0.193106,-0.189197,-0.206391,2222
3,-0.036595,-0.037734,-0.017077,-0.057986,-0.052252,-0.033158,-0.020228,-0.032603,-0.023876,-0.046224,...,-0.107888,-0.036940,-0.030034,-0.039882,-0.003321,0.006163,-0.007077,-0.047515,-0.045054,5484
4,0.090839,0.094680,0.101662,0.056721,0.042392,0.068095,0.101699,0.144664,0.140685,0.166099,...,0.194913,0.097767,0.161314,0.088485,0.165555,0.182740,0.265603,0.123040,0.132454,8686


In [13]:
train_data = pd.read_csv('C:/Users/Jorge Payà/Desktop/4Geeks/DSML Bootcamp/Projects/Regularized-Linear-model/data/processed/clean_train.csv')
test_data = pd.read_csv('C:/Users/Jorge Payà/Desktop/4Geeks/DSML Bootcamp/Projects/Regularized-Linear-model/data/processed/clean_test.csv')
train_data.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,70-79,80+,...,Total Specialist Physicians (2019),Total Population,Population Aged 60+,county_pop2018_18 and older,anycondition_number,Obesity_number,COPD_number,diabetes_number,CKD_number,Heart disease_number
0,-0.232556,-0.227731,-0.234284,-0.232951,-0.226353,-0.231316,-0.229599,-0.233425,-0.234680,-0.234420,...,-0.208590,-0.231195,-0.229737,-0.233171,-0.234370,-0.232975,-0.223516,-0.218609,-0.219329,2072
1,-0.158676,-0.178665,-0.180166,-0.188266,-0.175070,-0.161168,-0.134688,-0.105618,-0.119270,-0.091822,...,-0.110850,-0.150293,-0.098866,-0.152859,-0.142645,-0.155304,-0.110080,-0.131449,-0.130962,3796
2,-0.199114,-0.211128,-0.195138,-0.166782,-0.195036,-0.194045,-0.199725,-0.219256,-0.222207,-0.205154,...,-0.217668,-0.197005,-0.216056,-0.195125,-0.193205,-0.201976,-0.193106,-0.189197,-0.206391,2222
3,-0.036595,-0.037734,-0.017077,-0.057986,-0.052252,-0.033158,-0.020228,-0.032603,-0.023876,-0.046224,...,-0.107888,-0.036940,-0.030034,-0.039882,-0.003321,0.006163,-0.007077,-0.047515,-0.045054,5484
4,0.090839,0.094680,0.101662,0.056721,0.042392,0.068095,0.101699,0.144664,0.140685,0.166099,...,0.194913,0.097767,0.161314,0.088485,0.165555,0.182740,0.265603,0.123040,0.132454,8686


<span style="color: pink">We will use the train set to train the model, while with the test we will evaluate it to measure its degree of effectiveness. We will also split the predictors of the features.</span>

In [14]:
X_train = train_data.drop('Heart disease_number', axis=1) # create the feature matrix X by dropping the target variable
y_train = train_data['Heart disease_number'] # create the target variable y
X_test = test_data.drop('Heart disease_number', axis=1) # create the feature matrix X by dropping the target variable
y_test = test_data['Heart disease_number'] # create the target variable y

<strong><span style="color: red">Initialization and training of the models</span></strong>

In [16]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000) # create a logistic regression model
model.fit(X_train, y_train) # fit the model to the training data

LogisticRegression(max_iter=1000)

<span style="color: pink">After the training process we can know the parameters (variables a and b1, b2...bn) that the model has fitted:</span>

In [17]:
print(f"Intercept: {model.intercept_}")
print(f"Coefficients (b1, b2...bn): {model.coef_}")

Intercept: [-0.34636405 -0.34706254 -0.34376697 ... -7.24011258 -6.98566422
 -7.09964754]
Coefficients (b1, b2...bn): [[-0.08749314 -0.08288375 -0.08903423 ... -0.13277219 -0.0977491
  -0.09875411]
 [-0.0876655  -0.08355545 -0.08969575 ... -0.13280418 -0.09765227
  -0.09845675]
 [-0.08720874 -0.0827611  -0.08892303 ... -0.13238491 -0.09736575
  -0.09832386]
 ...
 [ 0.25893968  0.31854549  0.37259976 ...  0.36153972 -0.0290316
   0.30488731]
 [ 0.23625247  0.20854334  0.17082942 ...  0.35150069  0.17245313
   0.29997836]
 [ 0.27981822  0.20418708  0.21729391 ...  0.13827736  0.41429885
   0.303171  ]]


In [18]:
y_pred = model.predict(X_test) # make predictions on the testing data
y_pred

array([ 1072,  8689,  1072,  8689,  8689,  1072,  1072,  1072,  1072,
        1072,  1072,  1072,  3376,  1072,  7128,  1072, 75432,  1072,
        1072,  1072,  1072,  1072,  1072,  1072, 93262,  1072,  1072,
        1072,  1072,  1072,  1072,  1072,  1072,  1072,  8506,  1072,
        1072,  1072,  1072,  1072,  1072, 14102,  1072,  1072,  1072,
        1072,  1072,  3376,  1072,  1072, 32863,  1072,  1072,  1072,
        8506,  1072, 31550,  1072,  1072,  1072,  8689,  1072,  1072,
        8506,  1072,  1072,  1072,  1072,  1072, 23077,  1072,  1072,
        1072,  1072,  1072,  1448, 32863,  1072,  1072,  1072,  1072,
        3376, 12367,  1072,  1072,  1072, 32828,  1072,  1072,  7128,
        1072,  1072,  1072,  1072,  1072,  1072,  1072,  1072,  8506,
        1072, 43089,  8689,  1072,  1072,  1072,  1072,  1072,  1072,
        8506, 20299,  1072,  8506, 25091,  3376,  1072, 32828,  1072,
        1072,  8689,  1072,  1072,  1072,  1072,  1072,  1072,  1072,
        3376,  1072,

In [19]:
from sklearn.metrics import mean_squared_error, r2_score

print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"R-squared: {r2_score(y_test, y_pred)}")

MSE: 9968794.366242038
R-squared: 0.8861040840955449


<strong><span style="color: red">Model optimization</span></strong>

In [20]:
from sklearn.linear_model import Lasso

alpha = 1.0 # set the regularization strength
lasso_model = Lasso(alpha=alpha) # create a lasso model
lasso_model.fit(X_train, y_train) # fit the model to the training data

score = lasso_model.score(X_test, y_test) # calculate the R-squared score
print(f"Coefficients: {lasso_model.coef_}") # print the coefficients
print(f"R-squared: {score}") # print the R-squared score

Coefficients: [ 5103.56606854  1192.14607797 -1921.90787729  -804.66413704
  -565.56094295  4161.43524651   552.93901319 -1080.72356488
  3459.52199626  1245.55139019   999.98373671 -5424.05510818
   198.01474247  -841.91968637  -371.17714777  2792.66368052
   324.15462356    19.27250203 -1918.35380595    88.70731834
   492.63537754  -461.24851762  -854.99744188 -2893.16049233
  3359.53043536   204.58782867  1925.46994753  2907.60993035
  5383.37174712   819.64992462 -2329.43640877]
R-squared: 0.9978911816625889


c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.505e+08, tolerance: 7.097e+07
  model = cd_fast.enet_coordinate_descent(


In [22]:
from pickle import dump
dump(lasso_model, open('C:/Users/Jorge Payà/Desktop/4Geeks/DSML Bootcamp/Projects/Regularized-Linear-model/models/lasso_model.pkl', 'wb'))
# The .pkl extension is often used for files containing pickled data. The 'wb' argument specifies that the file is being opened for writing in binary mode

In [23]:
from sklearn.linear_model import Ridge

alpha = 1.0 # set the regularization strength
ridge_model = Ridge(alpha=alpha) # create a ridge model
ridge_model.fit(X_train, y_train) # fit the model to the training data

score = ridge_model.score(X_test, y_test) # calculate the R-squared score
print(f"Coefficients: {ridge_model.coef_}") # print the coefficients
print(f"R-squared: {score}") # print the R-squared score

Coefficients: [  -64.56359849  2950.33444089 -2303.19010741  -650.88542276
 -2145.88719238   696.65114622    47.01236968  -287.21607321
  2887.08428074   742.20719877  1771.22445761   -64.5635985
  1476.53623715   395.14135584  1739.19332979  1580.18795774
   548.65288753   638.78793769 -1556.32208098  -194.48515604
   406.97995994  -658.60642011  -658.85582258 -4526.68640677
   337.39971649    74.01819858  -160.98533231  2305.65748166
  5296.36667193  2483.88947159  2676.02061399]
R-squared: 0.9980896710652088


<div class="alert alert-block alert-success">
<b>Success:</b> By performing model optimization with regularized linear models we improve the model's prediction as we see in the result of R-squared score which provides the measure of how well the model's prediction match the actual values: 99% score. Finally, comparing between Lasso and Ridge, we note a nearly 1% enhancement favoring Ridge, which is a significant finding.</div>

In [ ]:
dump(ridge_model, open('C:/Users/Jorge Payà/Desktop/4Geeks/DSML Bootcamp/Projects/Regularized-Linear-model/models/ridge_model.pkl', 'wb'))